In [ ]:
function zero_or_x(pred_action, xs)
    if pred_action==1 then
        return xs
    else 
        return {0}
    end
end
function buildSummary(preds, xs, maxlen)
    local out = {}
    for i=1, #xs do
        if i == 1 then 
            out[i] = zero_or_x(preds[i], unpackZeros(xs[i]))
        else 
            --- Update it by adding xs_i and out_{i-1}
            out[i] = tableConcat(zero_or_x(preds[i], unpackZeros(xs[i])), out[i-1])
        end
    end
    if maxlen == 0 then
        maxlen = 0
        for k,v in pairs(out) do
            maxlen = math.max(maxlen, #v)
        end
    end
    outpadded = padZeros(out, maxlen)
    return outpadded
end

In [ ]:
-- out = buildSummary(preds, xs, 0)

# Starting model here

In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'
query_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv'

data_file = csvigo.load({path = aurora_fn, mode = "large"})
nugget_file = csvigo.load({path = nugget_fn, mode = "large"})
query_file =  csvigo.load({path = query_fn, mode = "large"})

rK = 500
batch_size = 500
nepochs = 10
print_every = 1
embed_dim = 10
learning_rate = 0.1

cuts = 4.                  --- This is the number of cuts we want
epsilon = 1.
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) --- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs

cuda = true
torch.manualSeed(420)

function build_network(vocab_size, embed_dim)
    local model = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
    :add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
    :add(nn.SelectTable(-1)) -- selects last state of the LSTM
    :add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification
    :add(nn.ReLU())
   return model
end

function build_model(vocab_size, embed_dim, outputSize)
    local mod1 = build_network(vocab_size, embed_dim)
    local mod2 = build_network(vocab_size, embed_dim)
    local mod3 = build_network(vocab_size, embed_dim)

    local mlp1 = nn.Sequential()
    mlp1:add(nn.Linear(1, embed_dim))
    mlp1:add(nn.ReLU())

    local ParallelModel = nn.ParallelTable()
    ParallelModel:add(mod1)
    ParallelModel:add(mod2)
    ParallelModel:add(mod3)
    ParallelModel:add(mlp1)

    local FinalMLP = nn.Sequential()
    FinalMLP:add(ParallelModel)
    FinalMLP:add(nn.JoinTable(2))
    FinalMLP:add( nn.Linear(embed_dim * 4, outputSize) )
    return FinalMLP
end

vocab_sized = getVocabSize(data_file)                       --- getting length of dictionary
vocab_sizeq = getVocabSize(query_file)                      --- getting length of dictionary
vocab_size = math.max(vocab_sized, vocab_sizeq)

queries = grabNsamples(query_file, #query_file-1, nil)      --- Extracting all queries
nuggets = grabNsamples(nugget_file, #nugget_file-1, nil)    --- Extracting all samples
maxseqlend = getMaxseq(data_file)                             --- Extracting maximum sequence length
maxseqlenq = getMaxseq(query_file)                            --- Extracting maximum sequence length
maxseqlen = math.max(maxseqlenq, maxseqlend)

batchLSTM = build_model(vocab_size, embed_dim, 1)
crit = nn.MSECriterion()

-- out = iterateModel( batch_size, nepochs, queries[3], data_file, batchLSTM, crit, epsilon, delta, 
--                     maxseqlen, base_explore_rate, print_every, nuggets, learning_rate, rK)


...Utils file loaded	


<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	


<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv	
<csv>	parsing done	


In [3]:
-- batch_size = batch_size
-- nepochs = nepochs
qs = queries[3]
x = data_file
model = batchLSTM
crit = crit 
-- epsilon
-- delta
mxl = maxseqlen
batch_size = 10
nepochs = 1
-- base_explore_rate = base_explore_rate
-- print_every = print_every
-- nuggets = nuggets
-- learning_rate = learning_rate
K = rK

In [4]:
rscores, pscores, fscores = {}, {}, {}
yrouge = torch.totable(torch.randn(#x))
summary_list = populateOnes(#x, K)
action_list = torch.totable(torch.round(torch.randn(#x)))
preds_list = torch.totable(torch.round(torch.rand(#x)))
print("training model...")
for epoch=0, nepochs, 1 do
    loss = 0                    --- Compute a new MSE loss each time
    --- Reset the rougue each time
     r_t1 , p_t1, f_t1 = 0., 0., 0.
    --- Looping over each bach of sentences for a given query
     nbatches = torch.floor( #x / batch_size)
    for minibatch = 1, nbatches do
        if minibatch == 1 then          -- Need +1 to skip the first row
            nstart = 2
            nend = torch.round(batch_size * minibatch)
        end
        if minibatch == nbatches then 
            nstart = nend + 1
            nend = #x
        end
        if minibatch > 1 and minibatch < nbatches then 
            nstart = nend + 1
            nend = torch.round(batch_size * minibatch)
        end
        --- This step is processing the data
         x_ss  = geti_n(x, nstart, nend)
         xout  = grabNsamples(x_ss, 1, #x_ss)     --- Extracting N samples
         xs  = padZeros(xout, mxl)                 --- Padding the data by the maximum length
         qs2 = padZeros({qs}, 5)
         qrep = repeatQuery(qs2[1], #xs)
         preds = geti_n(preds_list, nstart, nend)
         sumry_ss = buildSummary(preds, xs, 0)
         sentences = torch.LongTensor(xs):t()
         summary = torch.LongTensor(sumry_ss):t()
         query = torch.LongTensor(qrep):t()
         actions = torch.Tensor(geti_n(action_list, nstart, nend)):resize(#xs, 1)
         labels = torch.Tensor(geti_n(yrouge, nstart, nend))
        print('scoring...')
        myPreds = model:forward({sentences, summary, query, actions})
        print('...scored')
        loss = loss + crit:forward(myPreds, labels)
        grads = crit:backward(myPreds, labels)
        model:backward({sentences, summary, query, actions}, grads)
        model:updateParameters(learning_rate)        -- Update parameters after each minibatch
        model:zeroGradParameters()
        preds = policy(myPreds, epsilon, #xs)
        --- Concatenating predictions into a summary
        predsummary = buildPredSummary(preds, xs, K)
        --- Initializing rouge metrics at time {t-1} and save scores
        for i=1, #predsummary do
            --- Calculating rouge scores; Call get_i_n() to cumulatively compute rouge
            rscores[i] = rougeRecall(geti_n(predsummary, 1, i), nuggets, K) - r_t1
            pscores[i] = rougePrecision(geti_n(predsummary, 1, i), nuggets, K) - p_t1
            fscores[i] = rougeF1(geti_n(predsummary, 1, i), nuggets, K) - f_t1
            r_t1, p_t1, f_t1 = rscores[i], pscores[i], fscores[i]
        end
        --- Updating change in rouge
        yrouge = updateTable(yrouge, fscores, nstart)
        preds_list = updateTable(preds_list, preds, nstart)
        --- Calculating last one to see actual last rouge, without delta
        rscore = rougeRecall(predsummary, nuggets, K)
        pscore = rougePrecision(predsummary, nuggets, K)
        fscore = rougeF1(predsummary, nuggets, K)
        -- print(string.format('\t Mini-batch %i/%i', minibatch, nbatches) )
        break
        if (epoch % print_every)==0 then
             -- This line is useful to view the min and max of the predctions
            -- if epoch > 0 then  print(myPreds:min(), myPreds:max()) end
            perf_string = string.format(
                "Epoch %i, {Recall = %.6f, Precision = %.6f, F1 = %.6f}", 
                epoch, rscore, pscore, fscore
                )
            -- perf_string = string.format(
            --     "Epoch %i, sum(y)/len(y) = %i/%i, {Recall = %.6f, Precision = %.6f, F1 = %.6f}", 
            --     epoch, sumTable(preds_list), #preds_list, rscore, pscore, fscore
            --     )
            print(perf_string)
        end
    end
    epsilon = epsilon - delta           --- Decreasing the epsilon greedy strategy
    if epsilon <= 0 then                --- leave a random exploration rate
        epsilon = base_explore_rate
    end
end

training model...	


scoring...	


...scored	


scoring...	


...scored	


In [6]:
#summary

 44
  9
[torch.LongStorage of size 2]

